In [1]:
!pip install -q sentence-transformers langchain chromadb pypdf faiss-cpu \
langchain_community scikit-learn numpy mistralai langchain-mistralai


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.7/21.7 MB 68.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.2/328.2 kB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 76.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 95.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.8/442.8 kB 31.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.3/160.3 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 83.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 52.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 8.2 MB/s eta 

In [2]:
import os
from getpass import getpass

if not os.getenv("MISTRAL_API_KEY"):
    os.environ["MISTRAL_API_KEY"] = getpass("Enter your Mistral API Key: ")


Enter your Mistral API Key: ··········


In [3]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("/content/Sri_Harshith_Bollu_Resume (4).pdf")
documents = loader.load()

print(f"Loaded {len(documents)} pages")
print(documents[0].page_content[:300])


Loaded 1 pages
SRI HARSHITH BOLLU
IOS DEVELOPER
/ne+918328561927 sriharshit2000@gmail.com Bollu Sri Harshith
Summary
Results-driven IOS Developer with expertise in application development, UI/UX design, and software optimization.
Skilled in React, Python, Java, Swift, and SQL, with experience improving application


In [4]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100
)

chunks = splitter.split_documents(documents)
print(f"Total chunks created: {len(chunks)}")
print(chunks[0].page_content[:300])


Total chunks created: 7
SRI HARSHITH BOLLU
IOS DEVELOPER
/ne+918328561927 sriharshit2000@gmail.com Bollu Sri Harshith
Summary
Results-driven IOS Developer with expertise in application development, UI/UX design, and software optimization.
Skilled in React, Python, Java, Swift, and SQL, with experience improving application


In [5]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

vectorstore = Chroma.from_documents(
    documents=chunks,
    embedding=embeddings,
    persist_directory="./chroma_q2q"
)

retriever = vectorstore.as_retriever(search_kwargs={"k": 3})


/tmp/ipython-input-3887951989.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [6]:
user_query = "What does the document talk about?"

retrieved_docs = retriever.invoke(user_query)

for i, doc in enumerate(retrieved_docs):
    print(f"\n--- Chunk {i+1} ---")
    print(doc.page_content[:300])



--- Chunk 1 ---
capture detailed user journey flows within Digibank iOS apps.
• Led the implementation of an Auto Tagging Framework in Swift, reducing iOS developer time and effort by
92% and ensuring consistent analytics integration across multiple countries.
• Engineered a Performance Measurement library in Swift

--- Chunk 2 ---
SRI HARSHITH BOLLU
IOS DEVELOPER
/ne+918328561927 sriharshit2000@gmail.com Bollu Sri Harshith
Summary
Results-driven IOS Developer with expertise in application development, UI/UX design, and software optimization.
Skilled in React, Python, Java, Swift, and SQL, with experience improving application

--- Chunk 3 ---
defects, implementing unit tests that achieved 90% code coverage and significantly improved application
stability.
• Championed Agile development practices for iOS projects, utilizing Jira and Confluence to manage sprints and
iterative releases, resulting in a 15% boost in team productivity.
Senior 


In [7]:
from langchain_mistralai import ChatMistralAI
from langchain_core.prompts import ChatPromptTemplate

llm = ChatMistralAI(model="mistral-tiny")

q2q_prompt = ChatPromptTemplate.from_template("""
You are a query optimization assistant.

Rewrite the user query so that it retrieves the most relevant document chunks.
Make it specific and explicit.

User Query:
{query}

Optimized Query:
""")


In [15]:
optimized_query = llm.invoke(
    q2q_prompt.format(query=user_query)
).content

print("Original Query:", user_query)
print("Optimized Query:", optimized_query)


Original Query: What does the document talk about?
Optimized Query: "Provide a summary or identify the key topics discussed in the given document."


In [9]:
optimized_docs = retriever.invoke(optimized_query)

for i, doc in enumerate(optimized_docs):
    print(f"\n--- Optimized Chunk {i+1} ---")
    print(doc.page_content[:300])



--- Optimized Chunk 1 ---
capture detailed user journey flows within Digibank iOS apps.
• Led the implementation of an Auto Tagging Framework in Swift, reducing iOS developer time and effort by
92% and ensuring consistent analytics integration across multiple countries.
• Engineered a Performance Measurement library in Swift

--- Optimized Chunk 2 ---
performance using Firebase analytics and custom event tracking.
EDUCATION
Bachelor of Technology - Computer Science and Engineering 06 2018 – 04 2022
Institute of Aeronautical Engineering -CGPA - 8.63 Hyderabad, India
COURSEWORK / SKILLS
• IOS Development
• Application testing
• Mobile application
d

--- Optimized Chunk 3 ---
solutions that drive efficiency and innovation. A strong collaborator with experience working in Agile environments,
debugging, and integrating analytics to optimize performance. Recognized for delivering impactful solutions and
improving team productivity.
EXPERIENCE
Analyst 09 2023 – Present
DBS B


In [10]:
multi_hop_prompt = ChatPromptTemplate.from_template("""
Break the following question into multiple smaller sub-questions.
Each sub-question should retrieve different information.

Return them as a numbered list.

Question:
{query}
""")

sub_queries_text = llm.invoke(
    multi_hop_prompt.format(query=user_query)
).content

print(sub_queries_text)


1. Identify the main topic of the document: What is the primary subject or issue that the document is focusing on?

2. List secondary topics: What are the supporting or related topics that the document discusses in connection with the main topic?

3. Identify the document's purpose: What is the document's main objective or goal? Is it informative, persuasive, analytical, etc.?

4. Determine the document's structure: How is the document organized? Are there sections, subsections, or a specific format that the author follows?

5. Find the main arguments or ideas: What are the key points or arguments that the document presents to support its purpose?

6. Identify any supporting evidence or examples: Are there any facts, statistics, anecdotes, or case studies used to back up the main arguments or ideas?

7. Examine the tone and style: How does the author present the information? Is the tone formal or informal, objective or subjective, and what is the overall writing style?

8. Determine th

In [11]:
sub_queries = [
    q.strip("0123456789. ")
    for q in sub_queries_text.split("\n")
    if q.strip()
]

sub_queries


['Identify the main topic of the document: What is the primary subject or issue that the document is focusing on?',
 'List secondary topics: What are the supporting or related topics that the document discusses in connection with the main topic?',
 "Identify the document's purpose: What is the document's main objective or goal? Is it informative, persuasive, analytical, etc.?",
 "Determine the document's structure: How is the document organized? Are there sections, subsections, or a specific format that the author follows?",
 'Find the main arguments or ideas: What are the key points or arguments that the document presents to support its purpose?',
 'Identify any supporting evidence or examples: Are there any facts, statistics, anecdotes, or case studies used to back up the main arguments or ideas?',
 'Examine the tone and style: How does the author present the information? Is the tone formal or informal, objective or subjective, and what is the overall writing style?',
 'Determine the

In [16]:
all_retrieved_docs = []

for q in sub_queries:
    docs = retriever.invoke(q)
    all_retrieved_docs.extend(docs)

print(f"Total retrieved chunks: {len(all_retrieved_docs)}")


Total retrieved chunks: 33


In [17]:
unique_docs = {doc.page_content: doc for doc in all_retrieved_docs}
final_docs = list(unique_docs.values())

print(f"Unique chunks after fusion: {len(final_docs)}")


Unique chunks after fusion: 7


In [18]:
from langchain_classic.chains.combine_documents import create_stuff_documents_chain

answer_prompt = ChatPromptTemplate.from_template("""
Answer the question using ONLY the context below.
If the answer is not present, say "Not found in document".

<context>
{context}
</context>

Question:
{question}
""")

document_chain = create_stuff_documents_chain(llm, answer_prompt)

final_answer = document_chain.invoke({
    "context": final_docs,
    "question": user_query
})

print("\nFINAL ANSWER:\n")
print(final_answer)



FINAL ANSWER:

The document talks about a person named Sri Harshith Bollu, who is an iOS Developer, with a Bachelor's degree in Computer Science and Engineering from the Institute of Aeronautical Engineering in Hyderabad, India. He has technical skills in various programming languages such as Swift, Python, C++, JavaScript, Java, and SQL, and developer tools like VS Code, Android Studio, and XCode.

His professional experience includes working at DBS Bank as an Analyst and Senior Officer. In these roles, he has worked on various projects related to iOS development, including the integration of Swift-based native iOS screens with React modules, the development of in-house analytics SDKs in Swift, and the implementation of an Auto Tagging Framework in Swift for Firebase analytics and custom event tracking.

He has achieved several accomplishments such as the Extra Miler award from DBS Bank for delivering projects on time, reducing developer time and effort by 92% through the implementat